In [24]:
import os, av, PIL
from PIL import Image, ImageOps

In [25]:
videos_dirs = [os.path.join("../ARThermal",v) for v in os.listdir("../ARThermal") if v.startswith("videos_")]

# all videos stored on disk. dictionary with format {video_name:video_path, ...}
all_videos_paths = {}
for videos_dir in videos_dirs:
    groups = [os.path.join(videos_dir,g) for g in os.listdir(videos_dir) if os.path.isdir(os.path.join(videos_dir,g))]
    for g in groups:
        tims = os.listdir(g)
        if "14_tim" not in tims:
            continue
        
        videos_container = os.path.join(g, "14_tim")
        vids = [os.path.join(videos_container, v) for v in os.listdir(videos_container) if v.endswith(".mov")]
        for vid in vids:
            all_videos_paths[os.path.basename(vid)[:-4]] = vid

In [37]:
import json

poses= ["lying","sitting", "standing"]
frames = []
for pose in poses:
    with open("../ARThermal/dataset_merged_with_boxes/%s_frames_curated.json"%pose, "r") as f:
        frames.append(json.load(f))
    for frame in frames[-1]:
        frame["rects"][0]["x2"] = frame["rects"][0]["X2"]
        del frame["rects"][0]["X2"]
    with open("../ARThermal/dataset_merged_with_boxes/%s_frames_curated.json"%pose, "w") as f:
        json.dump(frames[-1], f)

In [38]:
"""return a PIL Image with a thermal frame if exists. otherwise return None"""
def get_thermal_frame_of_video(video_path, frame_id):
    try:
        container = av.open(video_path)
        video = next(s for s in container.streams if s.type == b'video')

        #super_frame_id is the frame that contains in one channel the frame_id
        super_frame_id = frame_id//3
        for packet in container.demux(video):
            for frame in packet.decode():
                if frame.index != super_frame_id:
                    continue

                img = frame.to_image()
                img = img.resize((640, 480))

                rgb = img.split()

                channel = rgb[frame_id%3]
                img2 = PIL.Image.merge("RGB", (channel, channel, channel))
                #img2 = ImageOps.mirror(img2)
                return img2
    except Exception as e:
        print "Error getting frames of video %s" % video_path
        print e
        return None

In [39]:
tensorbox_json_path = "../ARThermal/tensorbox_dataset_poses/tensorbox_input.json"
tensorbox_json = []
for cat_id, cat_frames in enumerate(frames):
    for frame in cat_frames:
        video_name = os.path.basename(os.path.dirname(frame["image_path"]))
        frame_name = os.path.basename(frame["image_path"])
        i,j = frame_name[:-4].split("_")

        frame_id = int(i)*3+int(j)
        img = get_thermal_frame_of_video(all_videos_paths[video_name], frame_id)
        
        img_dir = os.path.join("../ARThermal/tensorbox_dataset_poses", str(cat_id+1), video_name)
        if not os.path.isdir(img_dir):
            os.makedirs(img_dir)
            
        image_path = os.path.join(img_dir, frame_name)
        img.save(image_path)
        
        if frame["rects"][0].get("X2"):
            frame["rects"][0]["x2"] = frame["rects"][0]["X2"]
            del frame["rects"][0]["X2"]
            print "set x2"
        tensorbox_json.append({"image_path": image_path, "rects": frame["rects"]})

#import json
#with open(tensorbox_json_path, "w") as f:
#    json.dump(tensorbox_json, f)